<a href="https://colab.research.google.com/github/VENUGOPALADA/First-repo/blob/main/Drones_And_Bird.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.4/321.4 kB 21.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
pip install tensorflow

In [ ]:
import os
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import gradio as gr
import zipfile

dataset_path = '/content/Drone_And_Birds 01.zip'
extracted_folder_path = '/content/drones_birds_dataset'

with zipfile.ZipFile(dataset_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_folder_path)

dataset_files = os.listdir(extracted_folder_path)
print("Extracted files:", dataset_files)

In [ ]:


data_folder_path = os.path.join(extracted_folder_path, 'DronesBirdsDataset')
data_files = os.listdir(data_folder_path)

print("Contents of the dataset folder:", data_files)

train_path = os.path.join(data_folder_path, 'training')
test_path = os.path.join(data_folder_path, 'testing')

train_files = os.listdir(train_path)
test_files = os.listdir(test_path)

print("Training files:", train_files)
print("Testing files:", test_files)

# Define the model
Model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(16, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2, activation='softmax')  # Drones, Birds
])

Model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Data augmentation and rescaling
train_datagen = ImageDataGenerator(rescale=1.0/255.0, shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1.0/255.0)

train_generator = train_datagen.flow_from_directory(
    train_path,
    batch_size=32,
    class_mode='categorical',
    target_size=(150, 150)
)

test_generator = test_datagen.flow_from_directory(
    test_path,
    batch_size=32,
    class_mode='categorical',
    target_size=(150, 150)
)

# Train the model
history = Model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // 32,
    epochs=5,
    validation_data=test_generator,
    validation_steps=test_generator.samples // 32,
    verbose=2
)

# Prediction function
def classify_image(img):
    img = img.resize((150, 150))
    img_arr = image.img_to_array(img)
    img_arr = np.expand_dims(img_arr, axis=0)
    img_arr = img_arr / 255.0

    pred = Model.predict(img_arr)

    labels = ['Drone', 'Bird']  # Updated labels
    pred_class = labels[np.argmax(pred)]

    return f"Predicted Class: {pred_class}"

# Gradio interface
interface = gr.Interface(
    fn=classify_image,
    inputs=gr.Image(type="pil"),
    outputs=gr.Text(),
    live=True,
    title="Drone vs Bird Classifier",
    description="Upload an image, and the model will classify it as 'Drone' or 'Bird'."
)

interface.launch()
